# Tutorial (while using astropy)

follows SEP tutorial with image.fits from the SEP github

import numpy, matplotlib, astropy, and sep

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

In [ ]:
import sep

retrieve fits image

In [ ]:
fname = "image.fits"

read image as a 2-D array, and show image data

In [ ]:
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

display image

In [ ]:
m = np.mean(image_data)
s = np.std(image_data)
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

### Background Subtraction

a necessary step before sources can be detected

- with options to control box size, or mask pixels

In [ ]:
bkg = sep.Background(image_data)

"global" mean and noise of the background

In [ ]:
print(bkg,globalback)
print(bkg.globalrms)
#name 'globalback' is not defined

evaluate bkg as a 2d array

In [ ]:
bkg_image = bkg.back()

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

evaulate background noise as a 2d array

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

subtract the background from the image

In [ ]:
data_sub = image_data - bkg

### Object Detection

- detecting objects from the image without background noise

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

find how many objects were detected

In [ ]:
len(objects)

##### plot objects' (x,y) coordinates with basic parameters

plot background subtracted image

In [ ]:
fig, ax = plt.subplots()
m = np.mean(data_sub)
s = np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s,
               origin='lower')

plot an ellipse for each object

In [ ]:
from matplotlib.patches import Ellipse

In [ ]:
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]),
               width=6*objects['a'][i], height=6*objects['b'][i],
               angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

other available fields:

objects.dtype.names

### Aperture Photometry

circular aperture photometry with a 3 pixel radius at the locations of the objects

- where flux, fluxerr, and flag are 1-D arrays with one entry per object

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0,
                                    err=bkg.globalrms, gain=1.0)

show the first 10 objects' results

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))